# Energy Data Exploration

This notebook explores energy meter measurement from two different sources.

## Objectives
1. Load and understand the data structure
2. Group data by different time intervals (hourly, daily, weekly)
3. Identify patterns (day/night, weekday/weekend)
4. Form a hypothesis about what the data represents
5. Perform autocorrelation analysis

## Section 1: Data Loading

In [217]:
# Install required packages (run this cell first if packages are missing)
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [218]:
import pandas as pd
import requests
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt

# API endpoints (S3 JSON files simulating an API)
METER_1_URL = "https://exnaton-public-s3-bucket20230329123331528000000001.s3.eu-central-1.amazonaws.com/challenge/95ce3367-cbce-4a4d-bbe3-da082831d7bd.json"
METER_2_URL = "https://exnaton-public-s3-bucket20230329123331528000000001.s3.eu-central-1.amazonaws.com/challenge/1db7649e-9342-4e04-97c7-f0ebb88ed1f8.json"

print("Fetching data from API...")

Fetching data from API...


In [219]:
# Fetch data from both endpoints
meter_1 = requests.get(METER_1_URL)
meter_2 = requests.get(METER_2_URL)

import json
print(json.dumps(meter_1.json()["data"][0], indent=2))
print(json.dumps(meter_2.json()["data"][0], indent=2))



{
  "measurement": "energy",
  "timestamp": "2023-02-28T23:45:00.000Z",
  "tags": {
    "muid": "95ce3367-cbce-4a4d-bbe3-da082831d7bd",
    "quality": "measured"
  },
  "0100011D00FF": 0.0117
}
{
  "measurement": "energy",
  "timestamp": "2023-02-28T23:45:00.000Z",
  "tags": {
    "muid": "1db7649e-9342-4e04-97c7-f0ebb88ed1f8",
    "quality": "measured"
  },
  "0100021D00FF": 0
}


In [220]:

data_1 = meter_1.json()
data_2 = meter_2.json()

print(f"Meter 1 response status: {meter_1.status_code}")
print(f"Meter 2 response status: {meter_2.status_code}")

print("First few entries of data_1:")
print(pd.DataFrame(data_1['data']).head())

print("First few entries of data_2:")
print(pd.DataFrame(data_2['data']).head())

Meter 1 response status: 200
Meter 2 response status: 200
First few entries of data_1:
  measurement                 timestamp  \
0      energy  2023-02-28T23:45:00.000Z   
1      energy  2023-02-28T23:30:00.000Z   
2      energy  2023-02-28T23:15:00.000Z   
3      energy  2023-02-28T23:00:00.000Z   
4      energy  2023-02-28T22:45:00.000Z   

                                                tags  0100011D00FF  
0  {'muid': '95ce3367-cbce-4a4d-bbe3-da082831d7bd...        0.0117  
1  {'muid': '95ce3367-cbce-4a4d-bbe3-da082831d7bd...        0.0117  
2  {'muid': '95ce3367-cbce-4a4d-bbe3-da082831d7bd...        0.0118  
3  {'muid': '95ce3367-cbce-4a4d-bbe3-da082831d7bd...        0.0118  
4  {'muid': '95ce3367-cbce-4a4d-bbe3-da082831d7bd...        0.0210  
First few entries of data_2:
  measurement                 timestamp  \
0      energy  2023-02-28T23:45:00.000Z   
1      energy  2023-02-28T23:30:00.000Z   
2      energy  2023-02-28T23:15:00.000Z   
3      energy  2023-02-28T23:00:00.000Z

In [221]:
# Convert to DataFrames
# Data structure: dict with 'data' key containing list of measurements
# Each measurement has: measurement, timestamp, tags, and a value column (OBIS code)

df_1 = pd.DataFrame(data_1['data'])
df_2 = pd.DataFrame(data_2['data'])

# OBIS Code Structure for Energy Measurements
# These hex codes follow the OBIS standard: A-B-C-D-E-F
# 
# Meter 1:
# 0100011D00FF breakdown:
# - 01: Electricity (medium)
# - 00: Whole device/default channel
# - 01: Active power (actual energy consumed - what you get billed for)
# - 1D: Measurement type identifier
# - 00: Total energy across all tariffs
# - FF: Current value (255 = default/current)
#
# Meter 2:
# 0100021D00FF breakdown:
# - 01: Electricity (medium)
# - 00: Whole device/default channel
# - 02: Reactive power (energy for magnetic fields, not consumed but needed)
# - 1D: Measurement type identifier
# - 00: Total energy across all tariffs
# - FF: Current value (255 = default/current)
#
# In summary: These are measurements from the same device at the same time,
# capturing both active energy (01) and reactive energy (02).
#
# If beer is the active energy, then the reactive energy is the foam - you pay for the beer, not the foam.

# Rename OBIS code columns to standardized name and add meter labels
df_1['reading'] = df_1['0100011D00FF']
df_1['meter'] = 'Meter 1 (Active)'
df_1['measurement_type'] = 'active'
df_1 = df_1.drop(columns=['0100011D00FF'])

df_2['reading'] = df_2['0100021D00FF']
df_2['meter'] = 'Meter 2 (Reactive)'
df_2['measurement_type'] = 'reactive'
df_2 = df_2.drop(columns=['0100021D00FF'])

print(f"Meter 1 shape: {df_1.shape}")
print(f"Meter 2 shape: {df_2.shape}")
print(f"\nMeter 1 columns: {df_1.columns.tolist()}")
print(f"Meter 2 columns: {df_2.columns.tolist()}")

Meter 1 shape: (2688, 6)
Meter 2 shape: (2688, 6)

Meter 1 columns: ['measurement', 'timestamp', 'tags', 'reading', 'meter', 'measurement_type']
Meter 2 columns: ['measurement', 'timestamp', 'tags', 'reading', 'meter', 'measurement_type']


## Section 2: Data Understanding

In [222]:
# Check for missing values in each meter separately
print("=== Missing Values - Meter 1 (Active Energy) ===")
print(df_1.isnull().sum())
print(f"\nTotal rows: {len(df_1)}")

print("\n=== Missing Values - Meter 2 (Reactive Energy) ===")
print(df_2.isnull().sum())
print(f"\nTotal rows: {len(df_2)}")

=== Missing Values - Meter 1 (Active Energy) ===
measurement         0
timestamp           0
tags                0
reading             0
meter               0
measurement_type    0
dtype: int64

Total rows: 2688

=== Missing Values - Meter 2 (Reactive Energy) ===
measurement         0
timestamp           0
tags                0
reading             0
meter               0
measurement_type    0
dtype: int64

Total rows: 2688


In [223]:
# Convert timestamp to datetime
df_1['timestamp'] = pd.to_datetime(df_1['timestamp'])
df_2['timestamp'] = pd.to_datetime(df_2['timestamp'])

# Print timestamp range for both meters
print("\nMeter 1 timestamp range:")
print(f"Start: {df_1['timestamp'].min()}, End: {df_1['timestamp'].max()}")
print("\nMeter 2 timestamp range:")
print(f"Start: {df_2['timestamp'].min()}, End: {df_2['timestamp'].max()}")


Meter 1 timestamp range:
Start: 2023-02-01 00:00:00+00:00, End: 2023-02-28 23:45:00+00:00

Meter 2 timestamp range:
Start: 2023-02-01 00:00:00+00:00, End: 2023-02-28 23:45:00+00:00


In [224]:
# Combine both meter readings into a single DataFrame for easier analysis
df_combined = pd.concat([df_1, df_2], ignore_index=True)
df_combined = df_combined.sort_values('timestamp').reset_index(drop=True)

print(f"Combined DataFrame shape: {df_combined.shape}")
print(f"Columns: {df_combined.columns.tolist()}")
df_combined.head()

# Add time components for grouping
df_combined['hour'] = df_combined['timestamp'].dt.hour
df_combined['day_of_week'] = df_combined['timestamp'].dt.dayofweek  # 0=Monday, 6=Sunday
df_combined['day_name'] = df_combined['timestamp'].dt.day_name()
df_combined['date'] = df_combined['timestamp'].dt.date
df_combined['week'] = df_combined['timestamp'].dt.isocalendar().week
df_combined['week_of_month'] = ((df_combined['timestamp'].dt.day - 1) // 7) + 1  # Week 1-4 within month
df_combined['is_weekend'] = df_combined['day_of_week'].isin([5, 6])


Combined DataFrame shape: (5376, 6)
Columns: ['measurement', 'timestamp', 'tags', 'reading', 'meter', 'measurement_type']


In [225]:
# Show the first 3 rows of the combined DataFrame
df_combined.head(10)


,measurement,timestamp,tags,reading,meter,measurement_type,hour,day_of_week,day_name,date,week,week_of_month,is_weekend
0,energy,2023-02-01 00:00:00+00:00,{'muid': '95ce3367-cbce-4a4d-bbe3-da082831d7bd...,0.0162,Meter 1 (Active),active,0,2,Wednesday,2023-02-01,5,1,False
1,energy,2023-02-01 00:00:00+00:00,{'muid': '1db7649e-9342-4e04-97c7-f0ebb88ed1f8...,0.0000,Meter 2 (Reactive),reactive,0,2,Wednesday,2023-02-01,5,1,False
2,energy,2023-02-01 00:15:00+00:00,{'muid': '1db7649e-9342-4e04-97c7-f0ebb88ed1f8...,0.0000,Meter 2 (Reactive),reactive,0,2,Wednesday,2023-02-01,5,1,False
3,energy,2023-02-01 00:15:00+00:00,{'muid': '95ce3367-cbce-4a4d-bbe3-da082831d7bd...,0.0117,Meter 1 (Active),active,0,2,Wednesday,2023-02-01,5,1,False
4,energy,2023-02-01 00:30:00+00:00,{'muid': '95ce3367-cbce-4a4d-bbe3-da082831d7bd...,0.0118,Meter 1 (Active),active,0,2,Wednesday,2023-02-01,5,1,False
5,energy,2023-02-01 00:30:00+00:00,{'muid': '1db7649e-9342-4e04-97c7-f0ebb88ed1f8...,0.0000,Meter 2 (Reactive),reactive,0,2,Wednesday,2023-02-01,5,1,False
6,energy,2023-02-01 00:45:00+00:00,{'muid': '1db7649e-9342-4e04-97c7-f0ebb88ed1f8...,0.0000,Meter 2 (Reactive),reactive,0,2,Wednesday,2023-02-01,5,1,False
7,energy,2023-02-01 00:45:00+00:00,{'muid': '95ce3367-cbce-4a4d-bbe3-da082831d7bd...,0.0117,Meter 1 (Active),active,0,2,Wednesday,2023-02-01,5,1,False
8,energy,2023-02-01 01:00:00+00:00,{'muid': '1db7649e-9342-4e04-97c7-f0ebb88ed1f8...,0.0000,Meter 2 (Reactive),reactive,1,2,Wednesday,2023-02-01,5,1,False
9,energy,2023-02-01 01:00:00+00:00,{'muid': '95ce3367-cbce-4a4d-bbe3-da082831d7bd...,0.0117,Meter 1 (Active),active,1,2,Wednesday,2023-02-01,5,1,False


In [226]:
print("Min and Max values of 'reading' for each meter:")
min_max_reading = df_combined.groupby('meter')['reading'].agg(['min', 'max'])
print(min_max_reading)

Min and Max values of 'reading' for each meter:
                       min      max
meter                              
Meter 1 (Active)    0.0114  0.96120
Meter 2 (Reactive)  0.0000  0.17325


## Section 3: Visualization

In [227]:
# Raw time series visualization
# Each 'reading' is the energy (kWh) consumed/produced in that 15-minute interval
fig = px.line(
    df_combined, 
    x='timestamp', 
    y='reading',
    color='meter',
    title='Energy Data Time Series - Full Period',
    labels={'reading': 'Energy (kWh per 15-min)', 'timestamp': 'Date/Time'}
)
fig.update_layout(height=500)
fig.show()

In [228]:
# Hourly patterns (Day/Night analysis)
# Average of the 15-minute interval readings for each hour
hourly_avg = df_combined.groupby(['meter', 'hour'])['reading'].mean().reset_index()

fig = px.line(
    hourly_avg,
    x='hour',
    y='reading',
    color='meter',
    title='Average Energy by Hour of Day (Day/Night Pattern)',
    labels={'reading': 'Avg Energy (kWh per 15-min)', 'hour': 'Hour of Day (0-23)'}
)
fig.add_vrect(x0=6, x1=18, fillcolor='yellow', opacity=0.1, annotation_text='Daylight Hours')
fig.update_layout(height=400)
fig.show()

In [229]:
# Day of Week patterns
# Average of the 15-minute interval readings for each day of week
daily_avg = df_combined.groupby(['meter', 'day_of_week', 'day_name'])['reading'].mean().reset_index()
# Sort by day of week
daily_avg = daily_avg.sort_values('day_of_week')

fig = px.bar(
    daily_avg,
    x='day_name',
    y='reading',
    color='meter',
    barmode='group',
    title='Average Energy by Day of Week',
    labels={'reading': 'Avg Energy (kWh per 15-min)', 'day_name': 'Day'},
    category_orders={'day_name': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']}
)
fig.update_layout(height=400)
fig.show()

In [230]:
# Weekday vs Weekend comparison
# Average of the 15-minute interval readings for weekdays vs weekends
weekend_comparison = df_combined.groupby(['meter', 'is_weekend'])['reading'].agg(['mean', 'std', 'sum']).reset_index()
weekend_comparison['period'] = weekend_comparison['is_weekend'].map({True: 'Weekend', False: 'Weekday'})

fig = px.bar(
    weekend_comparison,
    x='period',
    y='mean',
    color='meter',
    barmode='group',
    error_y='std',
    title='Weekday vs Weekend Average Energy',
    labels={'mean': 'Avg Energy (kWh per 15-min)', 'period': 'Period'}
)
fig.update_layout(height=400)
fig.show()

In [231]:
# Daily totals over time
# Sum of all 96 readings (15-min intervals) per day = total kWh consumed/produced that day
daily_totals = df_combined.groupby(['meter', 'date'])['reading'].sum().reset_index()

fig = px.line(
    daily_totals,
    x='date',
    y='reading',
    color='meter',
    title='Daily Total Energy',
    labels={'reading': 'Total Energy (kWh)', 'date': 'Date'}
)
fig.update_layout(height=400)
fig.show()

In [232]:
# Weekly totals
# Sum of all readings for each week = total kWh consumed/produced that week
weekly_totals = df_combined.groupby(['meter', 'week_of_month'])['reading'].sum().reset_index()

fig = px.bar(
    weekly_totals,
    x='week_of_month',
    y='reading',
    color='meter',
    barmode='group',
    title='Weekly Total Energy (February 2023)',
    labels={'reading': 'Total Energy (kWh)', 'week_of_month': 'Week of Month'}
)
fig.update_layout(height=400, xaxis=dict(tickmode='linear', tick0=1, dtick=1))
fig.show()

In [233]:
# Heatmap: Hour of Day vs Day of Week
# Shows average 15-minute reading for each hour/day combination
for meter_name in df_combined['meter'].unique():
    meter_data = df_combined[df_combined['meter'] == meter_name]
    pivot = meter_data.pivot_table(
        values='reading', 
        index='hour', 
        columns='day_name', 
        aggfunc='mean'
    )
    # Reorder columns
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    pivot = pivot[[col for col in day_order if col in pivot.columns]]
    
    fig = px.imshow(
        pivot,
        title=f'Energy Heatmap: {meter_name}',
        labels=dict(x='Day of Week', y='Hour of Day', color='Avg (kWh/15-min)'),
        aspect='auto'
    )
    fig.update_layout(height=500)
    fig.show()

In [240]:
# Pattern Consistency Check: "What does each day of the week ACTUALLY look like?"
# 
# Instead of abstract autocorrelation numbers, let's SEE the patterns:
# - Overlay all 4 Mondays, all 4 Tuesdays, etc.
# - If lines stack on top of each other = consistent routine
# - If lines are all over the place = chaotic/unpredictable

import numpy as np

# Work with active energy (the main consumption)
df_active = df_combined[df_combined['measurement_type'] == 'active'].copy()
df_active['time_of_day'] = df_active['timestamp'].dt.hour + df_active['timestamp'].dt.minute / 60
df_active['week_num'] = df_active['timestamp'].dt.isocalendar().week

day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# ============================================================
# CHART 1: Overlay all weeks for each day (7 subplots)
# This shows: "How consistent is this person's Monday vs their Friday?"
# ============================================================

fig = make_subplots(
    rows=2, cols=4,
    subplot_titles=day_order + [''],
    specs=[[{}, {}, {}, {}], [{}, {}, {}, None]],
    horizontal_spacing=0.05,
    vertical_spacing=0.12
)

# Color palette for different weeks
week_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
weeks = sorted(df_active['week_num'].unique())

for idx, day_name in enumerate(day_order):
    row = 1 if idx < 4 else 2
    col = (idx % 4) + 1
    
    day_data = df_active[df_active['day_name'] == day_name]
    
    for week_idx, week in enumerate(weeks):
        week_day_data = day_data[day_data['week_num'] == week].sort_values('time_of_day')
        
        if len(week_day_data) > 0:
            fig.add_trace(
                go.Scatter(
                    x=week_day_data['time_of_day'],
                    y=week_day_data['reading'],
                    mode='lines',
                    name=f'Week {week_idx + 1}',
                    line=dict(color=week_colors[week_idx % len(week_colors)], width=1.5),
                    opacity=0.7,
                    showlegend=(idx == 0)  # Only show legend once
                ),
                row=row, col=col
            )

fig.update_layout(
    height=500,
    title_text='🔍 Pattern Consistency: Each Day Across All 4 Weeks',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='center', x=0.5)
)
fig.update_xaxes(title_text='Hour', tickvals=[0, 6, 12, 18, 24], row=2)
fig.update_yaxes(title_text='kWh', row=1, col=1)
fig.update_yaxes(title_text='kWh', row=2, col=1)
fig.show()

# ============================================================
# CHART 2: Average daily profile with confidence bands
# Shows the "typical" day with how much variation exists
# ============================================================

fig2 = go.Figure()

colors = {'Monday': '#1f77b4', 'Tuesday': '#ff7f0e', 'Wednesday': '#2ca02c', 
          'Thursday': '#d62728', 'Friday': '#9467bd', 'Saturday': '#8c564b', 'Sunday': '#e377c2'}

for day_name in day_order:
    day_data = df_active[df_active['day_name'] == day_name]
    
    # Group by time of day and calculate mean + std
    hourly_stats = day_data.groupby('hour')['reading'].agg(['mean', 'std']).reset_index()
    
    # Add confidence band (mean ± 1 std)
    fig2.add_trace(go.Scatter(
        x=list(hourly_stats['hour']) + list(hourly_stats['hour'][::-1]),
        y=list(hourly_stats['mean'] + hourly_stats['std']) + list((hourly_stats['mean'] - hourly_stats['std'])[::-1]),
        fill='toself',
        fillcolor=colors[day_name],
        opacity=0.1,
        line=dict(color='rgba(0,0,0,0)'),
        name=f'{day_name} range',
        showlegend=False
    ))
    
    # Add mean line
    fig2.add_trace(go.Scatter(
        x=hourly_stats['hour'],
        y=hourly_stats['mean'],
        mode='lines',
        name=day_name,
        line=dict(color=colors[day_name], width=2)
    ))


# ============================================================
# CONSISTENCY SCORE: How predictable is each day?
# ============================================================

print("=" * 70)
print("📈 CONSISTENCY SCORE BY DAY OF WEEK")
print("=" * 70)
print("(Lower coefficient of variation = more consistent/predictable routine)")
print()

consistency_scores = []
for day_name in day_order:
    day_data = df_active[df_active['day_name'] == day_name]
    
    # Calculate coefficient of variation for each hour, then average
    hourly_cv = day_data.groupby('hour')['reading'].agg(lambda x: x.std() / x.mean() if x.mean() > 0 else 0)
    avg_cv = hourly_cv.mean()
    consistency_scores.append({'day': day_name, 'cv': avg_cv, 'consistency': 1 - avg_cv})

consistency_df = pd.DataFrame(consistency_scores)
consistency_df = consistency_df.sort_values('consistency', ascending=False)

for _, row in consistency_df.iterrows():
    bar_len = int(row['consistency'] * 40)
    bar = '█' * bar_len + '░' * (40 - bar_len)
    emoji = '🏆' if row['consistency'] == consistency_df['consistency'].max() else '  '
    print(f"{emoji} {row['day']:<10} {bar} {row['consistency']:.1%}")

print()
print("🔍 INSIGHT: Most consistent day =", consistency_df.iloc[0]['day'])
print("           Least consistent day =", consistency_df.iloc[-1]['day'])

📈 CONSISTENCY SCORE BY DAY OF WEEK
(Lower coefficient of variation = more consistent/predictable routine)

🏆 Wednesday  █████████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ 24.4%
   Tuesday    █████░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ 14.3%
   Monday     ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ 7.3%
   Friday     ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ -1.2%
   Saturday   ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ -3.0%
   Thursday   ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ -6.4%
   Sunday     ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░ -21.6%

🔍 INSIGHT: Most consistent day = Wednesday
           Least consistent day = Sunday


## Section 4: Hypothesis Formation

From doing some basic internet sleuthing, I've came to the conclusion the hex values are OBIS codes. The description of which is explained above. The readings theory of active vs reactive energy tracks the most, as the percent of reactive energy is on average accurate against what the typical reactive energy is against active energy. In the real world, I would just go ask the provider/ or my fellow data science folk though. :)

- **Meter 1** (`0100011D00FF`): Active energy - the actual electricity being consumed (The beer you pay for)
- **Meter 2** (`0100021D00FF`): Reactive energy - the "phantom" power needed for magnetic fields in motors/transformers (The foam you don't pay for)

Sidenote - data is cool. But energy data is SO COOL! 

So, processing some more clues: energy heatmap for the start of the week (EARLY IN THE MORNING) is a crazy giveaway, when combined with the kwh numbers, that we're almost certainly looking at someone with a crazy consistent pattern, living in an appartment/ small house. The spikes are almost certainly related to them doing laundry. Also the low energy consumption friday evening is also a suggestion this user likes going out on those days, including sleeping in on saturday, hahaha.

Thinking about it further, I would go as far as to guess the person works on a hybrid schedule, where they usually go to office Tuesday/ Wednesday? Seems like their work day starts around 10 or so. And Wednesday they come back fairly early around 3? 

I just wish there was more data to try and get more trends from this. 

### The Verdict

We're looking at a **single person or couple in a small apartment/house** with:
- A consistent weekday routine (probably a hybrid work schedule with kid pickup responsibilities?)
- Regular laundry habits
- An active Friday night social calendar
- The self-respect to sleep in on Saturday and stay WAY TOO LATE on Sunday.